<a href="https://colab.research.google.com/github/visiont3lab/tecnologie_data_science/blob/master/book/docs/pyspark/logistic_regr_pyspark.ipynb
" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PYSPARK

In [1]:
################ template to run PySpark on Colab #######################

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark1 = SparkSession.builder.appName('basic').getOrCreate()
#Test must give no error

In [5]:
import pyspark

In [6]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("basic").setMaster("local")
#sc = SparkContext(conf=conf)  ## for jupyter and Databricks
sc = SparkContext.getOrCreate()   ## for Colab

In [7]:
from pyspark.sql.types import *

In [8]:
################ end template PySpark on Colab ##########################

### Breast Cancer Dataset

In [9]:
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/fav/tecno/breast_cancer.csv

cancer_df = spark.read.csv("breast_cancer.csv",header=True)
cancer_df.columns

--2020-06-22 21:46:04--  https://frenzy86.s3.eu-west-2.amazonaws.com/fav/tecno/breast_cancer.csv
Resolving frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)... 52.95.148.90
Connecting to frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)|52.95.148.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145599 (142K) [application/vnd.ms-excel]
Saving to: ‘breast_cancer.csv.2’

breast_cancer.csv.2 100%[===================>] 142.19K   193KB/s    in 0.7s    

2020-06-22 21:46:06 (193 KB/s) - ‘breast_cancer.csv.2’ saved [145599/145599]



['mean radius',
 'mean texture',
 'mean perimeter',
 'mean area',
 'mean smoothness',
 'mean compactness',
 'mean concavity',
 'mean concave points',
 'mean symmetry',
 'mean fractal dimension',
 'radius error',
 'texture error',
 'perimeter error',
 'area error',
 'smoothness error',
 'compactness error',
 'concavity error',
 'concave points error',
 'symmetry error',
 'fractal dimension error',
 'worst radius',
 'worst texture',
 'worst perimeter',
 'worst area',
 'worst smoothness',
 'worst compactness',
 'worst concavity',
 'worst concave points',
 'worst symmetry',
 'worst fractal dimension',
 'malignant']

Il DataFrame ha 31 colonne:
* 30 features: che rappresentano delle proprietà dell'immagine, come raggio, area e perimetro.
* 1 target: che è la colonna malignant, un valore di 1 indica un tumore maligno, al contrario un valore di 0 indica un tumore benigno.

In [10]:
cancer_df.show()

+-----------+------------+--------------+---------+-------------------+-------------------+-------------------+--------------------+-------------+----------------------+------------+------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+------------+-------------+---------------+----------+-------------------+-----------------+-------------------+--------------------+--------------+-----------------------+---------+
|mean radius|mean texture|mean perimeter|mean area|    mean smoothness|   mean compactness|     mean concavity| mean concave points|mean symmetry|mean fractal dimension|radius error|     texture error|   perimeter error|area error|    smoothness error|   compactness error|     concavity error|concave points error|      symmetry error|fractal dimension error|worst radius|worst texture|worst perimeter|worst area|   worst smoothness|worst compactness|    w

In [11]:
cancer_df.printSchema()

root
 |-- mean radius: string (nullable = true)
 |-- mean texture: string (nullable = true)
 |-- mean perimeter: string (nullable = true)
 |-- mean area: string (nullable = true)
 |-- mean smoothness: string (nullable = true)
 |-- mean compactness: string (nullable = true)
 |-- mean concavity: string (nullable = true)
 |-- mean concave points: string (nullable = true)
 |-- mean symmetry: string (nullable = true)
 |-- mean fractal dimension: string (nullable = true)
 |-- radius error: string (nullable = true)
 |-- texture error: string (nullable = true)
 |-- perimeter error: string (nullable = true)
 |-- area error: string (nullable = true)
 |-- smoothness error: string (nullable = true)
 |-- compactness error: string (nullable = true)
 |-- concavity error: string (nullable = true)
 |-- concave points error: string (nullable = true)
 |-- symmetry error: string (nullable = true)
 |-- fractal dimension error: string (nullable = true)
 |-- worst radius: string (nullable = true)
 |-- worst 

In [12]:
cancer_df.columns

['mean radius',
 'mean texture',
 'mean perimeter',
 'mean area',
 'mean smoothness',
 'mean compactness',
 'mean concavity',
 'mean concave points',
 'mean symmetry',
 'mean fractal dimension',
 'radius error',
 'texture error',
 'perimeter error',
 'area error',
 'smoothness error',
 'compactness error',
 'concavity error',
 'concave points error',
 'symmetry error',
 'fractal dimension error',
 'worst radius',
 'worst texture',
 'worst perimeter',
 'worst area',
 'worst smoothness',
 'worst compactness',
 'worst concavity',
 'worst concave points',
 'worst symmetry',
 'worst fractal dimension',
 'malignant']

In [13]:
data_schema = [StructField('mean radius', FloatType(), True),
                StructField('mean texture', FloatType(), True),
                StructField('mean perimeter', FloatType(), True),
                StructField('mean area', FloatType(), True),
                StructField('mean smoothness', FloatType(), True),
                StructField('mean compactness',FloatType(), True),
                StructField( 'mean concavity',FloatType(), True),
                StructField('mean concave points',FloatType(), True),
                StructField('mean symmetry',FloatType(), True),
                StructField('mean fractal dimension',FloatType(), True),
                StructField('radius error',FloatType(), True),
                StructField('texture error',FloatType(), True),
                StructField('perimeter error',FloatType(), True),
                StructField('area error',FloatType(), True),
                StructField('smoothness error',FloatType(), True),
                StructField('compactness error',FloatType(), True),
                StructField('concavity error',FloatType(), True),
                StructField('concave points error',FloatType(), True),
                StructField('symmetry error',FloatType(), True),
                StructField('fractal dimension error',FloatType(), True),
                StructField('worst radius',FloatType(), True),
                StructField('worst texture',FloatType(), True),
                StructField('worst perimeter',FloatType(), True),
                StructField('worst area',FloatType(), True),
                StructField('worst smoothness',FloatType(), True),
                StructField('worst compactness',FloatType(), True),
                StructField('worst concavity',FloatType(), True),
                StructField('worst concave points',FloatType(), True),
                StructField('worst symmetry',FloatType(), True),
                StructField('worst fractal dimension',FloatType(), True),
                StructField('malignant',FloatType(), True),]             
            
schema = StructType(fields=data_schema)

In [14]:
cancer_df = spark.read.csv("breast_cancer.csv",header=True,schema=schema)
cancer_df.columns

['mean radius',
 'mean texture',
 'mean perimeter',
 'mean area',
 'mean smoothness',
 'mean compactness',
 'mean concavity',
 'mean concave points',
 'mean symmetry',
 'mean fractal dimension',
 'radius error',
 'texture error',
 'perimeter error',
 'area error',
 'smoothness error',
 'compactness error',
 'concavity error',
 'concave points error',
 'symmetry error',
 'fractal dimension error',
 'worst radius',
 'worst texture',
 'worst perimeter',
 'worst area',
 'worst smoothness',
 'worst compactness',
 'worst concavity',
 'worst concave points',
 'worst symmetry',
 'worst fractal dimension',
 'malignant']

## Preprocessing dei dati
%md La classe MLlib richiede che le features si trovino tutte all'interno di un unico vettore su di una colonna, possiamo creare questa rappresentazione utilizzando la classe *VectorAssemlber* di MLlib:
* All'interno del parametro inputCols dobbiamo specificare quali sono le colonne con gli input.
* All'interno del parametro outputCol dobbiamo specificare il nome della colonna che conterrà le features.

In [15]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=cancer_df.columns[:-1], outputCol="features")
data_df = assembler.transform(cancer_df)

%md E' buona norma portare le features in un range di valori comuni, questo processo può velocizzare anche di molto la fase di addestramento. Facciamolo tramite la **standardizzazione** che ci permette di contenere le varie colonne all'interno di una distribuzione normale, cioè una distribuzione con media 0 e deviazione standard 1. Possiamo eseguire la standardizzazione usando la classe *StandardScaler* di MLlib.

In [16]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(data_df)
data_df = scaler_model.transform(data_df)

Ora possiamo creare i DataFrame per addestramento e test, estraendoli dal Dataframe originale, possiamo farlo tramite il metodo RandomSplit. Assegnamo il 70% degli esempi al set di addestramento e il 30% al set di test.

In [17]:
train_set, test_set = data_df.randomSplit([0.7, 0.3])

print("%d esempi nel train set" % train_set.count())
print("%d esempi nel test set" % test_set.count())

383 esempi nel train set
180 esempi nel test set


Ottimo ! Ora possiamo creare il modello di Regressione Logistica, usiamo la classe *LogisticRegression, all'interno del costruttore dovremo passare due parametri:
* **featuresCol**: il nome della colonna con le features
* **labelCol**: il nome della colonna con il target

In [18]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="scaled_features", labelCol="malignant")

Avviamo l'addestramento con il metodo *fit*, passando al suo interno il set di addetramento

In [19]:
model = lr.fit(train_set)

Abbiamo creato il nostro modello ! Ora verifichiamone la qualità.

## Valutiamo il Modello
Ora verifichiamone la qualità testandolo su dati che non ha visto durante l'addestramento, possiamo farlo usando il test set e il metodo *evalualte*.

In [20]:
evaluation = model.evaluate(test_set)

Il metodo *evaluate* calcolerà diverse metriche che ci possono aiutare a comprendere la qualità del modello, vediamone alcune.

#### Accuracy (Accuratezza)
L'accuracy indica semplicemente la percentuale di classificazioni che il nostro modello ha eseguito correttamente.

In [21]:
evaluation.accuracy

0.9444444444444444

### Precision (Precision)
La precision ci dice, tra le classificazioni eseguite per una data classe, quante sono effettivamente apparteneti a quella classe.

In [22]:
evaluation.precisionByLabel

[0.9180327868852459, 0.957983193277311]

### Recall (Richiamo)
Il recall ci dice quanti dei casi positivi il modello è riuscito a classificare correttamente.

In [23]:
evaluation.recallByLabel

[0.9180327868852459, 0.957983193277311]

## Testiamo il modello
Ora che abbiamo addestrato e validato il nostro modello, testiamolo su nuovi dati. Una clinica ci invia un file CSV contenente i risultati dell'agobiopsia per 3 pazienti che hanno in cura, dobbiamo utilizzare il nostro modello per identificare eventuali tumori maligni. Scarichiamo  il CSV

In [45]:
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/fav/tecno/exam_results.csv

--2020-06-22 21:51:31--  https://frenzy86.s3.eu-west-2.amazonaws.com/fav/tecno/exam_results.csv
Resolving frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)... 52.95.148.94
Connecting to frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)|52.95.148.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1143 (1.1K) [application/vnd.ms-excel]
Saving to: ‘exam_results.csv.1’

exam_results.csv.1  100%[===================>]   1.12K  --.-KB/s    in 0s      

2020-06-22 21:51:32 (28.2 MB/s) - ‘exam_results.csv.1’ saved [1143/1143]



In [46]:
exams_df = spark.read.csv("exam_results.csv", inferSchema=True, header=True)

Creiamo la colonna con le features

In [47]:
exams_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- mean radius: string (nullable = true)
 |-- mean texture: double (nullable = true)
 |-- mean perimeter: double (nullable = true)
 |-- mean area: double (nullable = true)
 |-- mean smoothness: double (nullable = true)
 |-- mean compactness: double (nullable = true)
 |-- mean concavity: double (nullable = true)
 |-- mean concave points: double (nullable = true)
 |-- mean symmetry: double (nullable = true)
 |-- mean fractal dimension: double (nullable = true)
 |-- radius error: double (nullable = true)
 |-- texture error: double (nullable = true)
 |-- perimeter error: double (nullable = true)
 |-- area error: double (nullable = true)
 |-- smoothness error: double (nullable = true)
 |-- compactness error: double (nullable = true)
 |-- concavity error: double (nullable = true)
 |-- concave points error: double (nullable = true)
 |-- symmetry error: double (nullable = true)
 |-- fractal dimension error: double (nullable = true)
 |-- worst radius: d

In [48]:
data_schema = [StructField('id', IntegerType(), True),
                StructField('mean radius', FloatType(), True),
                StructField('mean texture', FloatType(), True),
                StructField('mean perimeter', FloatType(), True),
                StructField('mean area', FloatType(), True),
                StructField('mean smoothness', FloatType(), True),
                StructField('mean compactness',FloatType(), True),
                StructField( 'mean concavity',FloatType(), True),
                StructField('mean concave points',FloatType(), True),
                StructField('mean symmetry',FloatType(), True),
                StructField('mean fractal dimension',FloatType(), True),
                StructField('radius error',FloatType(), True),
                StructField('texture error',FloatType(), True),
                StructField('perimeter error',FloatType(), True),
                StructField('area error',FloatType(), True),
                StructField('smoothness error',FloatType(), True),
                StructField('compactness error',FloatType(), True),
                StructField('concavity error',FloatType(), True),
                StructField('concave points error',FloatType(), True),
                StructField('symmetry error',FloatType(), True),
                StructField('fractal dimension error',FloatType(), True),
                StructField('worst radius',FloatType(), True),
                StructField('worst texture',FloatType(), True),
                StructField('worst perimeter',FloatType(), True),
                StructField('worst area',FloatType(), True),
                StructField('worst smoothness',FloatType(), True),
                StructField('worst compactness',FloatType(), True),
                StructField('worst concavity',FloatType(), True),
                StructField('worst concave points',FloatType(), True),
                StructField('worst symmetry',FloatType(), True),
                StructField('worst fractal dimension',FloatType(), True)]             
            
schema = StructType(fields=data_schema)

In [49]:
exams_df = spark.read.csv("exam_results.csv", schema=schema, header=True)

In [50]:
exams_df.show(5)

+------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+
|    id|mean radius|mean texture|mean perimeter|mean area|mean smoothness|mean compactness|mean concavity|mean concave points|mean symmetry|mean fractal dimension|radius error|texture error|perimeter error|area error|smoothness error|compactness error|concavity error|concave points error|symmetry error|fractal dimension error|worst radius|worst texture|worst perimeter|worst area|worst smoothness|worst compactness|worst concavity|worst concave points|worst symmetry|worst fractal dimension|
+------+----

In [55]:
new_data_df = assembler.transform(exams_df)

Eseguiamo la standardizzazione

In [57]:
new_data_df = scaler_model.transform(new_data_df)

E otteniamo le predizioni usando il metodo *transform* del modello

In [58]:
pred_df = model.transform(new_data_df)
pred_df.columns

['id',
 'mean radius',
 'mean texture',
 'mean perimeter',
 'mean area',
 'mean smoothness',
 'mean compactness',
 'mean concavity',
 'mean concave points',
 'mean symmetry',
 'mean fractal dimension',
 'radius error',
 'texture error',
 'perimeter error',
 'area error',
 'smoothness error',
 'compactness error',
 'concavity error',
 'concave points error',
 'symmetry error',
 'fractal dimension error',
 'worst radius',
 'worst texture',
 'worst perimeter',
 'worst area',
 'worst smoothness',
 'worst compactness',
 'worst concavity',
 'worst concave points',
 'worst symmetry',
 'worst fractal dimension',
 'features',
 'scaled_features',
 'rawPrediction',
 'probability',
 'prediction']

Come vedi il DataFrame risultante contiene due nuove colonne:
* **Prediction**: che contiene il label (0=benigno, 1=maligno)
* **Probabily**: che contiene la probabilità di apparteneza alle due classi.

In [ ]:
#pred_df.select(["probability", "prediction"]).show(6, False)
pred_df.select("probability", "prediction").show()


Due dei tumori sono stati etichettati come maligni, con delle probabilità associate molto alte, sarebbe il caso che la clinica effettuasse ulteriori esami.